### Slade Brooks
##### brooksl@mail.uc.edu
##### 02.03.2023
# AeroComputing Assignment 2



# Part 1

In [2]:
# import shtuff
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas
import sys

# import custom modules
sys.path.append("..//lib//")
import pyvot        # pivot direct methods
import pyterative       # iterative new cool epic methods

# Part 2

In [10]:
# define a and b
a = np.zeros((20, 20), dtype="float64")
for i in range(20):
    a[i, i] = 4.
    if i < 19:
        a[i, i+1] = -1.
        a[i+1, i] = -1.
a[0, 19] = 1.
a[19, 0] = 1.

b = np.zeros((20, 1), dtype="float64")
b[19, 0] = 100.

In [ ]:
# compute each of direct methods

In [11]:
# define interEqs fxn for gauss
def iterEqs(x, omega):
    n = len(x)
    x[0] = omega*(x[1] - x[n-1])/4.0 + (1.0 - omega)*x[0]
    for i in range(1, n-1):
        x[i] = omega*(x[i-1] + x[i+1])/4.0 + (1.0 - omega)*x[i]
    x[n-1] = omega*(100. - x[0] + x[n-2])/4.0 \
        + (1.0 - omega)*x[n-1]
    return x

# do gaussSeidel method
x = np.zeros(20)        # because n = 20
x,numIter,omega = pyterative.gaussSeidel(iterEqs,x)
print("\nNumber of iterations =",numIter)
print("\nRelaxation factor =",omega)
print("\nThe solution is:\n",x)


Number of iterations = 21

Relaxation factor = 1.097679755833418

The solution is:
 [-7.73502692e+00 -2.07259421e+00 -5.55349941e-01 -1.48805549e-01
 -3.98722562e-02 -1.06834753e-02 -2.86164518e-03 -7.63105381e-04
 -1.90776345e-04  8.65000954e-14  1.90776346e-04  7.63105381e-04
  2.86164518e-03  1.06834753e-02  3.98722562e-02  1.48805549e-01
  5.55349941e-01  2.07259421e+00  7.73502692e+00  2.88675135e+01]
